In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

%matplotlib inline
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

hp = pd.read_csv("./data/HousePrice2.csv", index_col = 0)
hp = hp.dropna()
hp.shape

In [ ]:
htmlColors = pd.read_csv("./data/html_colors.csv", 
                         skiprows = 7,
                        names = ['name1','name2','hex', 'r','g','b'])
htmlColors.head()



In [ ]:
len(hp['Neighborhood'].unique())

In [ ]:
with open ('./data/GmapAPI.txt') as f: 
    api_key = f.readline()
    f.close

In [ ]:
import gmaps
gmaps.configure(api_key=api_key)

In [ ]:
locator = Nominatim(user_agent="hi")
ISU = locator.geocode('Ames, IA 50011')
ISU_coord = (ISU.latitude, ISU.longitude)
ISU_coord

In [ ]:
address = hp[['latitude', 'longitude','Neighborhood']]

neighbors = hp.Neighborhood.unique()
np.random.seed(0)
colors = np.random.choice(htmlColors['hex'], size = len(hp['Neighborhood'].unique()), \
                          replace = False)
nbDict = dict(zip(neighbors,colors))

address['Neighborhood_color'] = hp['Neighborhood'].map(nbDict)

address = address.dropna()
address.head()

In [ ]:
fig = gmaps.figure(center = ISU_coord, map_type = 'TERRAIN', zoom_level = 12,
                  layout = {'width': '1000px', 'height': '1000px'})
#neighborhood = hp['Neighborhood'].map[]
markers = gmaps.symbol_layer(
    address[['latitude','longitude']], 
    hover_text = list(address['Neighborhood']),
    fill_color = list(address['Neighborhood_color']),
    fill_opacity = 1,
    stroke_opacity = 0.0,
    scale = 2
)

fig.add_layer(markers)
fig


In [ ]:
fig = gmaps.figure(center = ISU_coord, zoom_level = 12, 
                   layout = {'width': '1000px', 'height': '1000px'})
#neighborhood = hp['Neighborhood'].map[]
markers = gmaps.symbol_layer(
    address[['latitude','longitude']], 
    hover_text = list(address['Neighborhood']),
    fill_color = list(address['Neighborhood_color']),
    fill_opacity = 1,
    stroke_opacity = 0.0,
    scale = 2
)

fig.add_layer(markers)
fig



In [ ]:
nbDict2 = dict(zip(colors,neighbors))
htmlColors['Neighborhood'] = htmlColors['hex'].map(nbDict2)
htmlColors.dropna()


In [ ]:
hp_res= hp[hp['MSZoning'].isin(['FV','RH','RL','RM'])]
a_res = hp_res[['latitude', 'longitude','Neighborhood']]

neighbors = hp.Neighborhood.unique()
np.random.seed(0)
colors = np.random.choice(htmlColors['hex'], size = len(hp['Neighborhood'].unique()), \
                          replace = False)
nbDict = dict(zip(neighbors,colors))

a_res['Neighborhood_color'] = a_res['Neighborhood'].map(nbDict)

a_res = address.dropna()
a_res.head()

In [ ]:
fig = gmaps.figure(center = ISU_coord, zoom_level = 12,
                   layout = {'width': '1000px', 'height': '1000px'})
#neighborhood = hp['Neighborhood'].map[]
markers = gmaps.symbol_layer(
    a_res[['latitude','longitude']], 
    hover_text = list(a_res['Neighborhood']),
    fill_color = list(a_res['Neighborhood_color']),
    fill_opacity = 1,
    stroke_opacity = 0.0,
    scale = 2
)

fig.add_layer(markers)
fig

In [ ]:
hp_res.YearBuilt.sort_values(ascending = False)

In [ ]:
hp_res['price_per_sqft'] = hp_res['SalePrice']/hp_res['GrLivArea']
hp_res['price_per_sqft_q'] = pd.qcut(hp_res['price_per_sqft'], 3,     \
                                     labels = ['low value', 'medium value','high value'])
hp_res['SalePrice_q'] = pd.qcut(hp_res['SalePrice'], 3,      \
                              labels = ['low','medium','high'])

def label_value(row):
    if row['price_per_sqft_q']=='high value' and \
       row['SalePrice_q']=='high': 
        return 'high'
    elif row['price_per_sqft_q']=='low value' and \
       row['SalePrice_q']=='low': 
        return 'low'
    else:
        return 'medium'
hp_res['value'] = hp_res.apply(lambda row: label_value(row), axis = 1)

In [ ]:
a_res = hp_res[['latitude', 'longitude','Neighborhood','value',
               'SalePrice', 'GrLivArea','price_per_sqft','YearBuilt']]

value = hp_res.value.unique()
np.random.seed(26)
colors = np.random.choice(htmlColors['hex'], size = len(hp_res['value'].unique()), \
                          replace = False)
valueDict = dict(zip(value,colors))

a_res['value_color'] = a_res['value'].map(valueDict)
a_res = a_res.dropna()
a_res.value_color.unique()

In [ ]:
fig = gmaps.figure(center = ISU_coord, zoom_level = 12,
                 layout = {'width': '1000px', 'height': '1000px'})
#neighborhood = hp['Neighborhood'].map[]
markers = gmaps.symbol_layer(
    a_res[['latitude','longitude']], 
    hover_text = list(a_res['Neighborhood']),
    fill_color = list(a_res['value_color']),
    fill_opacity = 1,
    stroke_opacity = 0.0,
    scale = 2
)

fig.add_layer(markers)
fig

In [ ]:
hp_res['YearBuilt'].loc[1] >1945

In [ ]:
a_res = hp_res[['latitude', 'longitude','Neighborhood','YearBuilt']]
a_res['YearBuilt'] = a_res['YearBuilt']
def label_age(row):
    if row['YearBuilt']<1945: 
        return 'Pre-War'
    elif row['YearBuilt'] <2000: 
        return 'Old'
    else:
        return 'New'
a_res['age'] = hp_res.apply(lambda row: label_age(row), axis = 1)
a_res.head()

In [ ]:
a_res['age']

In [ ]:
np.random.seed(8)
colors = np.random.choice(htmlColors['hex'], size = len(a_res['age'].unique()), \
                          replace = False)
age = a_res.age.unique()
ageDict = dict(zip(age,colors))

a_res['age_color'] = a_res['age'].map(ageDict)
a_res = a_res.dropna()
a_res.age_color.unique()
ageDict

In [ ]:
fig = gmaps.figure(center = ISU_coord, zoom_level = 12,
                 layout = {'width': '1000px', 'height': '1000px'})
#neighborhood = hp['Neighborhood'].map[]
markers = gmaps.symbol_layer(
    a_res[['latitude','longitude']], 
    hover_text = list(a_res['age']),
    fill_color = list(a_res['age_color']),
    fill_opacity = 1,
    stroke_opacity = 0.0,
    scale = 2
)

fig.add_layer(markers)
fig